In [14]:
# from ipyfilechooser import FileChooser
from netCDF4 import Dataset
import numpy.ma as ma
import numpy as np
from ipyleaflet import Polyline, Map, Marker, basemap_to_tiles, basemaps, AwesomeIcon

In [15]:
# Select Argo file
# projDir = 'Argo_data/'
# fc = FileChooser(projDir)
# display(fc)

In [16]:
# argo_fn = fc.selected
argo_fn = 'Argo_data//2902730_prof.nc'

In [17]:
# Open NetCDF Argo file
nc_fid = Dataset(argo_fn,'r')
vars = nc_fid.variables.keys()
vars

dict_keys(['DATA_TYPE', 'FORMAT_VERSION', 'HANDBOOK_VERSION', 'REFERENCE_DATE_TIME', 'DATE_CREATION', 'DATE_UPDATE', 'PLATFORM_NUMBER', 'PROJECT_NAME', 'PI_NAME', 'STATION_PARAMETERS', 'CYCLE_NUMBER', 'DIRECTION', 'DATA_CENTRE', 'DC_REFERENCE', 'DATA_STATE_INDICATOR', 'DATA_MODE', 'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE', 'JULD', 'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE', 'POSITION_QC', 'POSITIONING_SYSTEM', 'PROFILE_PRES_QC', 'PROFILE_TEMP_QC', 'PROFILE_PSAL_QC', 'VERTICAL_SAMPLING_SCHEME', 'CONFIG_MISSION_NUMBER', 'PRES', 'PRES_QC', 'PRES_ADJUSTED', 'PRES_ADJUSTED_QC', 'PRES_ADJUSTED_ERROR', 'TEMP', 'TEMP_QC', 'TEMP_ADJUSTED', 'TEMP_ADJUSTED_QC', 'TEMP_ADJUSTED_ERROR', 'PSAL', 'PSAL_QC', 'PSAL_ADJUSTED', 'PSAL_ADJUSTED_QC', 'PSAL_ADJUSTED_ERROR', 'PARAMETER', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 'SCIENTIFIC_CALIB_DATE', 'HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_SOFTWARE', 'HISTORY_SOFTW

In [18]:
lat = nc_fid.variables['LATITUDE'][:]
lon = nc_fid.variables['LONGITUDE'][:]
dateJD = nc_fid.variables['JULD'][:]
date_ref = nc_fid.variables['REFERENCE_DATE_TIME'][:]
nc_fid.close()

In [19]:
# Reference datetime
dd=date_ref.tobytes()
dd=dd.decode('utf-8')
ddnp = np.datetime64(dd[0:4]+'-'+dd[4:6]+'-'+dd[6:8]+'T'+dd[8:10]+':'+dd[10:12]+':'+dd[12:14])
print(ddnp)

1950-01-01T00:00:00


In [20]:
date = ddnp + np.array(dateJD//1,dtype='timedelta64[D]') + np.array((dateJD%1)*24*60*60,dtype='timedelta64[s]')
# date

In [21]:
nProf = lat.count()
# nProf

In [22]:
latitude = lat.reshape(nProf,1)
longitude = lon.reshape(nProf,1)
coords = ma.concatenate([latitude, longitude],axis=1)
# coords

In [23]:
argoTraj_line = Polyline(
    locations=coords.tolist(),
    color="black" ,
    fill=False,
    weight=3
)

In [24]:
from ipywidgets import Label
label = Label()
display(label)
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        m_coords = kwargs.get('coordinates')
        m_coords[0] = round(m_coords[0],5)
        m_coords[1] = round(m_coords[1],5)
        label.value = str(m_coords[0])+'  '+str(m_coords[1])

#
center = (lat.mean(), lon.mean())
m = Map(basemap=basemap_to_tiles(basemaps.Esri.NatGeoWorldMap), center=center, zoom=5)
m.add_layer(argoTraj_line);
for i in range(0,nProf):
    marker = Marker(location = coords[i].tolist(),draggable=False)
    marker.title = 'Profile '+str(i+1)+'\n'+str(date[i])
    marker.rise_on_hover = True
    marker.icon = AwesomeIcon(name='share',marker_color='beige',icon_color='black',spin=False)
    m.add_layer(marker)
m.layout.width = '100%'
m.layout.height = '500px'
m.on_interaction(handle_interaction)
m

Label(value='')

Map(center=[12.967607843137257, 127.63063725490196], controls=(ZoomControl(options=['position', 'zoom_in_text'…